# Quantum Chemistry: Potential Energy Surfaces

In this notebook, we compute potential energy surfaces (PES) for small molecules using the Variational Quantum Eigensolver (VQE). We compare VQE results against exact classical solutions (full configuration interaction) to assess accuracy.

Molecules studied:
1. **H2** — bond distance scan from 0.3 to 2.5 Angstroms
2. **LiH** — bond distance scan from 0.5 to 4.0 Angstroms
3. **Ansatz comparison** — RealAmplitudes vs EfficientSU2 on H2

The PySCF driver computes molecular integrals, and Jordan-Wigner mapping converts the fermionic Hamiltonian to qubit operators.

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from src.molecules import build_h2, build_lih, build_beh2
from src.vqe_runner import run_vqe
from src.exact_solver import exact_energy
from src.pes_scan import scan_pes
from src.plotting import (
    plot_pes, plot_energy_error,
    plot_multi_molecule_comparison, plot_ansatz_comparison
)

%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

RESULTS_DIR = Path('../results')
RESULTS_DIR.mkdir(exist_ok=True)

SEED = 42

print('Setup complete!')

## 1. Single-Point VQE on H2

Before scanning the full PES, let's verify VQE works at a single geometry — the equilibrium bond distance of H2 (0.735 Angstroms).

In [ ]:
# build H2 Hamiltonian at equilibrium
h2_op, h2_problem = build_h2(distance=0.735)
nuc_repulsion = h2_problem.nuclear_repulsion_energy

print(f'H2 at d=0.735 A:')
print(f'  Number of qubits: {h2_op.num_qubits}')
print(f'  Number of Pauli terms: {len(h2_op)}')
print(f'  Nuclear repulsion: {nuc_repulsion:.6f} Ha')

# run VQE
vqe_result = run_vqe(h2_op, ansatz_type='RealAmplitudes', seed=SEED)
vqe_total = vqe_result['energy'] + nuc_repulsion

# exact reference
exact_e = exact_energy(h2_op) + nuc_repulsion

error = abs(vqe_total - exact_e)
print(f'\n  VQE energy:   {vqe_total:.8f} Ha')
print(f'  Exact energy: {exact_e:.8f} Ha')
print(f'  Error:        {error*1000:.4f} mHa')
print(f'  Chemical accuracy (1.6 mHa): {"YES" if error*1000 < 1.6 else "NO"}')

In [ ]:
# plot VQE convergence
fig, ax = plt.subplots(figsize=(10, 5))

history = [e + nuc_repulsion for e in vqe_result['energy_history']]
ax.plot(history, '-', color='#FF6B6B', linewidth=1.5, alpha=0.8)
ax.axhline(y=exact_e, color='#2C3E50', linestyle='--', linewidth=1.5,
           label=f'Exact = {exact_e:.6f} Ha')

ax.set_xlabel('VQE Iteration')
ax.set_ylabel('Energy (Ha)')
ax.set_title('VQE Convergence for H2 at Equilibrium')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'h2_convergence.png', dpi=150)
plt.show()

## 2. H2 Potential Energy Surface

Now we scan the H-H bond distance from 0.3 to 2.5 Angstroms. At short distances the nuclei repel strongly (energy goes up), at the equilibrium distance (~0.735 A) the energy is minimized, and at large distances the molecule dissociates.

The dissociation region is interesting because it has strong electron correlation — methods like Hartree-Fock fail there, but VQE (being variational) should track the exact curve.

In [ ]:
# scan H2 PES
h2_distances = np.linspace(0.3, 2.5, 15)

print('Scanning H2 PES...')
h2_results = scan_pes(
    build_h2, h2_distances,
    ansatz_type='RealAmplitudes', reps=3,
    maxiter=200, seed=SEED
)

print(f'\nMean error: {np.mean(h2_results["errors"])*1000:.3f} mHa')
print(f'Max error:  {np.max(h2_results["errors"])*1000:.3f} mHa')

In [ ]:
# plot H2 PES
plot_pes(h2_results, 'H2', save_dir=str(RESULTS_DIR))

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# PES curve
ax = axes[0]
ax.plot(h2_results['distances'], h2_results['exact_energies'], 'o-',
        color='#2C3E50', linewidth=2, markersize=6, label='Exact (FCI)')
ax.plot(h2_results['distances'], h2_results['vqe_energies'], 's--',
        color='#FF6B6B', linewidth=2, markersize=6, alpha=0.8, label='VQE')
ax.set_xlabel('Bond Distance (A)')
ax.set_ylabel('Energy (Ha)')
ax.set_title('H2 Potential Energy Surface')
ax.legend()
ax.grid(True, alpha=0.3)

# error plot
ax = axes[1]
errors_mha = [e * 1000 for e in h2_results['errors']]
ax.plot(h2_results['distances'], errors_mha, 'o-', color='#E74C3C',
        linewidth=2, markersize=6)
ax.axhline(y=1.6, color='#2ECC71', linestyle='--', linewidth=1.5,
           alpha=0.7, label='Chemical accuracy (1.6 mHa)')
ax.set_xlabel('Bond Distance (A)')
ax.set_ylabel('|VQE - Exact| (mHa)')
ax.set_title('H2 VQE Error')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(RESULTS_DIR / 'h2_pes_and_error.png', dpi=150)
plt.show()

# find equilibrium distance
min_idx = np.argmin(h2_results['exact_energies'])
print(f'Equilibrium distance: {h2_results["distances"][min_idx]:.3f} A')
print(f'Equilibrium energy: {h2_results["exact_energies"][min_idx]:.6f} Ha')

## 3. LiH Potential Energy Surface

LiH is a more challenging molecule — it has more electrons and orbitals than H2, and the dissociation region shows stronger multi-reference character. We use an active space reduction to keep the qubit count manageable.

In [ ]:
# scan LiH PES
lih_distances = np.linspace(0.5, 4.0, 12)

print('Scanning LiH PES...')
lih_results = scan_pes(
    build_lih, lih_distances,
    ansatz_type='RealAmplitudes', reps=3,
    maxiter=200, seed=SEED
)

print(f'\nMean error: {np.mean(lih_results["errors"])*1000:.3f} mHa')
print(f'Max error:  {np.max(lih_results["errors"])*1000:.3f} mHa')

In [ ]:
# plot LiH PES
plot_pes(lih_results, 'LiH', save_dir=str(RESULTS_DIR))
plot_energy_error(lih_results, 'LiH', save_dir=str(RESULTS_DIR))

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

ax = axes[0]
ax.plot(lih_results['distances'], lih_results['exact_energies'], 'o-',
        color='#2C3E50', linewidth=2, markersize=6, label='Exact (FCI)')
ax.plot(lih_results['distances'], lih_results['vqe_energies'], 's--',
        color='#4ECDC4', linewidth=2, markersize=6, alpha=0.8, label='VQE')
ax.set_xlabel('Bond Distance (A)')
ax.set_ylabel('Energy (Ha)')
ax.set_title('LiH Potential Energy Surface')
ax.legend()
ax.grid(True, alpha=0.3)

ax = axes[1]
errors_mha = [e * 1000 for e in lih_results['errors']]
ax.plot(lih_results['distances'], errors_mha, 'o-', color='#E74C3C',
        linewidth=2, markersize=6)
ax.axhline(y=1.6, color='#2ECC71', linestyle='--', linewidth=1.5,
           alpha=0.7, label='Chemical accuracy (1.6 mHa)')
ax.set_xlabel('Bond Distance (A)')
ax.set_ylabel('|VQE - Exact| (mHa)')
ax.set_title('LiH VQE Error')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(RESULTS_DIR / 'lih_pes_and_error.png', dpi=150)
plt.show()

## 4. Multi-Molecule Comparison

Let's put the H2 and LiH PES curves side by side to compare how well VQE tracks the exact solution for different molecules.

In [ ]:
all_results = {
    'H2': h2_results,
    'LiH': lih_results,
}

plot_multi_molecule_comparison(all_results, save_dir=str(RESULTS_DIR))

# inline side-by-side
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for idx, (name, results) in enumerate(all_results.items()):
    ax = axes[idx]
    ax.plot(results['distances'], results['exact_energies'], 'o-',
            color='#2C3E50', linewidth=2, markersize=5, label='Exact')
    ax.plot(results['distances'], results['vqe_energies'], 's--',
            color=['#FF6B6B', '#4ECDC4'][idx], linewidth=2,
            markersize=5, alpha=0.8, label='VQE')
    ax.set_xlabel('Bond Distance (A)')
    ax.set_ylabel('Energy (Ha)')
    ax.set_title(f'{name} PES')
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)

plt.suptitle('VQE Potential Energy Surfaces', fontsize=14, y=1.02)
plt.tight_layout()
plt.savefig(RESULTS_DIR / 'comparison_h2_lih.png', dpi=150, bbox_inches='tight')
plt.show()

print('\nSummary:')
for name, results in all_results.items():
    mean_err = np.mean(results['errors']) * 1000
    max_err = np.max(results['errors']) * 1000
    print(f'  {name}: mean error = {mean_err:.3f} mHa, '
          f'max error = {max_err:.3f} mHa')

## 5. Ansatz Comparison on H2

We compare two ansatz types on the H2 PES:
- **RealAmplitudes**: Ry rotations + linear CNOT entanglement, fewer parameters
- **EfficientSU2**: Full single-qubit rotations (Ry, Rz) + circular CNOT, more expressive

More expressive ansatze can represent a wider range of quantum states but may be harder to optimize (more parameters, potential barren plateaus).

In [ ]:
# scan with EfficientSU2
print('Scanning H2 with EfficientSU2 ansatz...')
h2_esu2_results = scan_pes(
    build_h2, h2_distances,
    ansatz_type='EfficientSU2', reps=3,
    maxiter=200, seed=SEED
)

print(f'\nEfficientSU2 mean error: {np.mean(h2_esu2_results["errors"])*1000:.3f} mHa')
print(f'RealAmplitudes mean error: {np.mean(h2_results["errors"])*1000:.3f} mHa')

In [ ]:
# comparison plot
ansatz_results = {
    'RealAmplitudes': h2_results,
    'EfficientSU2': h2_esu2_results,
}

plot_ansatz_comparison(ansatz_results, 'H2', save_dir=str(RESULTS_DIR))

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

ax = axes[0]
ax.plot(h2_distances, h2_results['exact_energies'], 'o-',
        color='#2C3E50', linewidth=2, markersize=5, label='Exact')
ax.plot(h2_distances, h2_results['vqe_energies'], 's--',
        color='#FF6B6B', linewidth=2, markersize=5, alpha=0.8,
        label='RealAmplitudes')
ax.plot(h2_distances, h2_esu2_results['vqe_energies'], '^--',
        color='#4ECDC4', linewidth=2, markersize=5, alpha=0.8,
        label='EfficientSU2')
ax.set_xlabel('Bond Distance (A)')
ax.set_ylabel('Energy (Ha)')
ax.set_title('H2 PES: Ansatz Comparison')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

ax = axes[1]
ax.plot(h2_distances, [e*1000 for e in h2_results['errors']], 'o-',
        color='#FF6B6B', linewidth=2, markersize=6, label='RealAmplitudes')
ax.plot(h2_distances, [e*1000 for e in h2_esu2_results['errors']], 's-',
        color='#4ECDC4', linewidth=2, markersize=6, label='EfficientSU2')
ax.axhline(y=1.6, color='#2ECC71', linestyle='--', linewidth=1.5,
           alpha=0.7, label='Chemical accuracy')
ax.set_xlabel('Bond Distance (A)')
ax.set_ylabel('|VQE - Exact| (mHa)')
ax.set_title('H2 Error: Ansatz Comparison')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(RESULTS_DIR / 'h2_ansatz_comparison.png', dpi=150)
plt.show()

## Summary

### What we found:

1. **VQE accurately reproduces the H2 PES** — including the dissociation region where Hartree-Fock fails due to strong correlation. The VQE error stays well within chemical accuracy across the full range of bond distances.

2. **LiH is more challenging** — the larger Hilbert space and stronger correlations near dissociation lead to slightly larger VQE errors, but the PES shape is still captured accurately.

3. **Ansatz choice matters** — both RealAmplitudes and EfficientSU2 work well for H2, but their relative performance can differ at different geometries. More expressive ansatze don't always win because they're harder to optimize.

4. **Active space reduction is essential** — for LiH and larger molecules, reducing to an active space keeps the qubit count manageable while capturing the essential physics.

### Relevance:

PES calculations are fundamental to computational chemistry — they determine reaction barriers, equilibrium structures, and molecular properties. VQE's ability to handle strong correlation (unlike Hartree-Fock) and scale to larger systems (unlike exact diagonalization) makes it a promising tool for quantum chemistry on near-term quantum hardware.